# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
from peft import LoraConfig, get_peft_model, PeftModelForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding, default_data_collator
from datasets import load_dataset, load_metric

model_name = 'distilbert-base-uncased'

/opt/conda/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


In [2]:
dataset = load_dataset('ag_news')
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.add_special_tokens({'pad_token': '[PAD]', 'eos_token' : '[EOS]'})

def preprocess_function(examples):
    encodings = tokenizer(examples['text'], padding='max_length', truncation=True)
    encodings["labels"] = examples["label"]
    return encodings

encoded_dataset = dataset.map(preprocess_function, batched=True)
encoded_dataset.set_format(type="torch", columns=['input_ids', 'attention_mask', 'labels'])

small_train_dataset = encoded_dataset["train"].shuffle(seed=42).select([i for i in list(range(200))])
small_eval_dataset = encoded_dataset["test"].shuffle(seed=42).select([i for i in list(range(50))])



Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [5]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4)
model.resize_token_embeddings(len(tokenizer))

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_eval_batch_size=8,
    per_device_train_batch_size=8,
    logging_steps=10,
    save_total_limit=1,
    logging_dir="./log"
)

metric = load_metric("accuracy")

def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(axis=1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
    compute_metrics=compute_metrics,
)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'pre_classifier.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [6]:
eval_results = trainer.evaluate()
print("Evaluation results before fine-tuning:", eval_results)

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Evaluation results before fine-tuning: {'eval_loss': 1.3772064447402954, 'eval_accuracy': 0.32, 'eval_runtime': 83.2568, 'eval_samples_per_second': 0.601, 'eval_steps_per_second': 0.084}


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [10]:
config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    inference_mode = False,
    target_modules=["q_lin", "k_lin", "v_lin", "out_lin"],
    modules_to_save=["classifier"],
)


In [11]:
lora_model = get_peft_model(model, config)
lora_model.print_trainable_parameters()
trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
    compute_metrics=compute_metrics,
)

lora_model.save_pretrained('distilbert-lora')

trainable params: 1,777,160 || all params: 68,140,808 || trainable%: 2.6080700422572036


## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [12]:
lora_model = PeftModelForSequenceClassification.from_pretrained(model, 'distilbert-lora')

In [13]:
trainer = Trainer(
    model=lora_model,
    args=training_args,
    eval_dataset=small_eval_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
    compute_metrics=compute_metrics,
)


In [14]:
eval_results_peft = trainer.evaluate()
print("Evaluation results after fine-tuning:", eval_results_peft)

Evaluation results after fine-tuning: {'eval_loss': 1.3772064447402954, 'eval_accuracy': 0.32, 'eval_runtime': 92.8287, 'eval_samples_per_second': 0.539, 'eval_steps_per_second': 0.075}
